In [ ]:
#default_exp main

In [ ]:
import os
import pandas as pd
import numpy as np
import transformers

from fastai.text.all import *

from Hasoc import utils
from Hasoc import model

In [ ]:
#export 
MODEL_NAME = 'bert-base-uncased'
MAX_SEQ_LEN = 72
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
#export
df = pd.read_csv(os.path.join('../data', 'en_task_a', 'hasoc_2020_en_train_new.csv'), sep='\t')

In [ ]:
#hide
df.sample(2)

,tweet_id,text,task1,task2,ID
2850,1.126784e+18,@xbuttercupally @AmeliaMelody_ shit is creepy tbh,HOF,PRFN,hasoc_2020_en_2822
1198,1.130103e+18,@LunchBox507 @BlueStrike905 @graceabelles @FallenIntrovrt @sunnibunnn I have never worked on my birthday. Nor gone to school. 😁,NOT,NONE,hasoc_2020_en_5358


In [ ]:
#hide
df['task1'].value_counts()

HOF    1856
NOT    1852
Name: task1, dtype: int64

In [ ]:
#hide
df['task2'].value_counts()

NONE    1852
PRFN    1377
OFFN     321
HATE     158
Name: task2, dtype: int64

In [ ]:
#export
df.tweet_id = df.tweet_id.astype('int').astype('str')

In [ ]:
df = df[:1000]

In [ ]:
#export
bert_tokenizer = transformers.BertTokenizer.from_pretrained(
            pretrained_model_name_or_path=MODEL_NAME,
            do_lower_case=True,
            )

In [ ]:
bert_vocab = bert_tokenizer.get_vocab()
bert2fastai_vocab = [k for k, v in sorted(bert_vocab.items(), key=lambda item: item[1])]

In [ ]:
#export 
pad = partial(pad_input_chunk, pad_first=False, pad_idx=bert_tokenizer.pad_token_id, seq_len=MAX_SEQ_LEN)

In [ ]:
#export
custom_tokenizer = Tokenizer.from_df(text_cols='text', tok=utils.HFTokenizer(tokenizer=bert_tokenizer), rules=[])

In [ ]:
#export
tfms = [attrgetter('text'), custom_tokenizer, Numericalize(vocab=bert2fastai_vocab), utils.Add_Special_Cls(tokenizer=bert_tokenizer)]

In [ ]:
#export
splits = RandomSplitter()(df)

In [ ]:
#export
dsets = Datasets(df, tfms=[tfms, [attrgetter('task1'), Categorize()]], splits=splits, dl_type=SortedDL)

In [ ]:
#export
dls_clas = dsets.dataloaders(bs=56, val_bs=144, before_batch=[pad], seq_len=MAX_SEQ_LEN)

In [ ]:
#export
dls_clas.show_batch(max_n=2)

,text,category
0,[CLS] rt @ sud ##out ##ats ##uy ##a : eat shit grammar nazis ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! [SEP],HOF
1,[CLS] rt @ er ##ot ##ik _ center : mail order dom ##ina ##trix part 5 - liza calls him “ mr . smith ” and tells him she … ! & gt ; & gt ; https : / / t . co / k ##d ##gm ##n ##j ##0 ##th ##r & lt ; & lt ; https : / / t . co … [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD],NOT


In [ ]:
#export
dsets.c

2

In [ ]:
#export
model = model.HasocModel(out_feat=2)

In [ ]:
#export
learn = Learner(dls_clas, model.to(device), metrics=[accuracy, F1Score(average='macro')], drop_mult=0.3, device=device, loss_func=CrossEntropyLossFlat())

In [ ]:
#export
learn.lr_find()

KeyboardInterrupt: 

In [ ]:
learn.validate()

(#3) [0.7882494330406189,0.4950000047683716,0.33110367892976583]